In [4]:
from transformers import AutoModel, AutoTokenizer
import torch
import torch.nn.functional as F

# Load PhoBERT model and tokenizer from Hugging Face
model = AutoModel.from_pretrained("vinai/phobert-base-v2")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

# Define query and documents
query = "vía hẻ"
docs = ["lề đường"]

# Function to get embeddings
def get_embedding(text):
    # Tokenize and convert to PyTorch tensors
    encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    
    # Get model output
    with torch.no_grad():
        model_output = model(**encoded_input)
    
    # Use the [CLS] token embedding as the sentence embedding
    sentence_embedding = model_output.last_hidden_state[:, 0, :]
    
    # Normalize the embedding
    sentence_embedding = F.normalize(sentence_embedding, p=2, dim=1)
    
    return sentence_embedding

# Encode query and documents
query_embedding = get_embedding([query])
doc_embeddings = torch.cat([get_embedding([doc]) for doc in docs])

# Calculate cosine similarity
similarities = torch.mm(query_embedding, doc_embeddings.transpose(0, 1)).flatten()

# Sort documents by cosine similarity
sorted_indices = torch.argsort(similarities, descending=True)
sorted_docs = [docs[idx] for idx in sorted_indices]
sorted_scores = [similarities[idx].item() for idx in sorted_indices]

# Print sorted documents with their cosine scores
for doc, score in zip(sorted_docs, sorted_scores):
    print(f"Document: {doc} - Cosine Similarity: {score:.4f}")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Document: lề đường - Cosine Similarity: 0.5847
